# Part 2: Training

In this part we will show how using MLRun's **Feature Store** we can easily define a **Feature Vector** and create the dataset we need to run our training process.  
By the end of this tutorial you’ll learn how to:
- Combine multiple data sources to a single Feature Vector
- Create training dataset
- Create a model using an MLRun Hub function

In [1]:
project_name = 'fraud-demo'

In [2]:
import mlrun

# Initialize the MLRun project object
project = mlrun.get_or_create_project(project_name, context="./", user_project=True)

> 2023-01-25 10:43:32,631 [info] loaded project fraud-demo from MLRun DB


## Step 1 - Create a Feature Vector  
In this section we will create our Feature Vector.  
The Feature vector will have a `name` so we can reference to it later via the URI or our serving function, and a list of `features` from the available FeatureSets.  We can add a feature from a feature set by adding `<FeatureSet>.<Feature>` to the list, or add `<FeatureSet>.*` to add all the FeatureSet's available features.  

By default, the first FeatureSet in the feature list will act as the spine. meaning that all the other features will be joined to it.  
For example, in this instance we use the early sense sensor data as our spine, so for each early sense event we will create produce a row in the resulted Feature Vector.

In [3]:
# Define the list of features we will be using
features = ['transactions.*',
            'events.*']

In [4]:
# Import MLRun's Feature Store
import mlrun.feature_store as fstore

# Define the feature vector name for future reference
fv_name = 'transactions-fraud'

# Define the feature vector using our Feature Store (fstore)
transactions_fv = fstore.FeatureVector(fv_name,
                          features, 
                          label_feature="labels.label",
                          description='Predicting a fraudulent transaction')

# Save the feature vector in the Feature Store
transactions_fv.save()

## Step 2 - Preview the Feature Vector Data

Obtain the values of the features in the feature vector, to ensure the data appears as expected

In [5]:
# Import the Parquet Target so we can directly save our dataset as a file
from mlrun.datastore.targets import ParquetTarget

# Get offline feature vector as dataframe and save the dataset to parquet
train_dataset = fstore.get_offline_features(fv_name, target=ParquetTarget())

> 2023-01-25 10:43:44,359 [info] wrote target: {'name': 'parquet', 'kind': 'parquet', 'path': 'v3io:///projects/fraud-demo-dani/FeatureStore/transactions-fraud/parquet/vectors/transactions-fraud-latest.parquet', 'status': 'ready', 'updated': '2023-01-25T10:43:44.359239+00:00', 'size': 2664931, 'partitioned': True}


In [6]:
# Preview our dataset
train_dataset.to_dataframe().tail(5)

,amount_avg_12h,amount_sum_12h,amount_count_12h,amount_max_12h,es_transportation_count_14d,es_health_count_14d,es_otherservices_count_14d,es_food_count_14d,es_hotelservices_count_14d,es_barsandrestaurants_count_14d,...,category_es_travel,category_es_leisure,gender_M,gender_F,gender_E,timestamp_window,event_details_change,event_login,event_password_change,label
125973,32.515517,942.95,29,104.03,29.0,29.0,29.0,29.0,29.0,29.0,...,0,0,1,0,0,14d,1.0,0.0,0.0,0
125974,30.424348,699.76,23,64.79,23.0,23.0,23.0,23.0,23.0,23.0,...,0,0,1,0,0,14d,0.0,0.0,1.0,0
125975,27.320526,519.09,19,56.05,19.0,19.0,19.0,19.0,19.0,19.0,...,0,0,0,1,0,14d,0.0,0.0,1.0,0
125976,84.328929,2361.21,28,1506.61,28.0,28.0,28.0,28.0,28.0,28.0,...,0,0,1,0,0,14d,1.0,0.0,0.0,0
125977,11.350000,68.10,6,20.70,6.0,6.0,6.0,6.0,6.0,6.0,...,0,0,0,1,0,14d,0.0,1.0,0.0,0


## Step 3 - Train Models and Choose Highest Accuracy

With MLRun, one can easily train different models and compare the results. In the code below, we train 3 different models,
each uses a different algorithm (random forest, XGBoost, adabost), and choose the model with the highest accuracy

In [ ]:
# Import the Sklearn classifier function from the functions hub
classifier_fn = mlrun.import_function('hub://sklearn-classifier')

In [ ]:
# Prepare the parameters list for the training function
# We will be using 3 different models
training_params = {"model_name": ['transaction_fraud_rf', 
                                  'transaction_fraud_xgboost', 
                                  'transaction_fraud_adaboost'],
              
              "model_pkg_class": ['sklearn.ensemble.RandomForestClassifier',
                                  'sklearn.ensemble.GradientBoostingClassifier',
                                  'sklearn.ensemble.AdaBoostClassifier']}

# Define the training task, including our feature vector, label and hyperparams definitions
train_task = mlrun.new_task('training', 
                      inputs={'dataset': transactions_fv.uri},
                      params={'label_column': 'label'}
                     )

train_task.with_hyper_params(training_params, strategy='list', selector='max.accuracy')

# Specify our cluster image
classifier_fn.spec.image = 'mlrun/mlrun'

# Run training
classifier_fn.run(train_task, local=False)

## Step 4 - Perform Feature Selection

As part of our data science process we will try and reduce the training dataset's size to get rid of bad or unuseful features and save computation time.

We will use our ready-made feature selection function from our hub [`hub://feature_selection`](https://github.com/mlrun/functions/blob/development/feature_selection/feature_selection.ipynb) to select the best features to keep on a sample from our dataset and run the function on that.


In [ ]:
feature_selection_fn = mlrun.import_function('hub://feature_selection')

feature_selection_run = feature_selection_fn.run(
            params={'sample_ratio':0.25,
                    'output_vector_name':fv_name + "-short",
                   'ignore_type_errors': True},
    
            inputs={'df_artifact': transactions_fv.uri},
            name='feature_extraction',
            handler='feature_selection',
    local=False)

In [ ]:
mlrun.get_dataitem(feature_selection_run.outputs['top_features_vector']).as_df().tail(5)

## Step 5 - Train our models with top features

Following the feature selection, we train new models using the resultant features. We can observe the accuracy and other results remain high
meaning we get a model that requires less features to be accurate and thus less error-prone.

In [ ]:
# Defining our training task, including our feature vector, label and hyperparams definitions
ensemble_train_task = mlrun.new_task('training', 
                      inputs={'dataset': feature_selection_run.outputs['top_features_vector']},
                      params={'label_column': 'label'}
                     )
ensemble_train_task.with_hyper_params(training_params, strategy='list', selector='max.accuracy')

classifier_fn.run(ensemble_train_task)

## Done!

You've completed Part 2 of the model training with the feature store.
Proceed to [Part 3](03-deploy-serving-model.ipynb) to learn how to deploy and monitor the model.